In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
from datetime import datetime

import folium
import xarray as xr
from folium import plugins
from ocean_navigation_simulator.env.data_sources.DrifterData import DrifterData
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import lux

dataset_drifter = {
    'host': 'nrt.cmems-du.eu',#ftp host => nrt.cmems-du.eu for Near Real Time products
    'product': 'INSITU_GLO_UV_NRT_OBSERVATIONS_013_048',#name of the In Situ Near Real Time product in the GLO area
    'name': 'drifter',# name of the dataset available in the above In Situ Near Real Time product
    'index_files': ['index_latest.txt', 'index_monthly.txt', 'index_history.txt'], #files describing the content of the lastest, monthly and history netCDF file collections available withint he above dataset
    'index_platform': 'index_platform.txt', #files describing the network of platforms contributing with files in the above collections
}
dataset_argo = {
    'host': 'nrt.cmems-du.eu',#ftp host => nrt.cmems-du.eu for Near Real Time products
    'product': 'INSITU_GLO_UV_NRT_OBSERVATIONS_013_048',#name of the In Situ Near Real Time product in the GLO area
    'name': 'argo',# name of the dataset available in the above In Situ Near Real Time product
    'index_files': ['index_history.txt'], #files describing the content of the lastest, monthly and history netCDF file collections available withint he above dataset
}

# define initial area of interest in order to increase speed
targeted_geospatial_lat_min = 17.0  # enter min latitude of your bounding box
targeted_geospatial_lat_max = 32.0  # enter max latitude of your bounding box
targeted_geospatial_lon_min = -120.0 # enter min longitude of your bounding box
targeted_geospatial_lon_max = -140.0 # enter max longitude of your bounding box
targeted_bbox = [targeted_geospatial_lon_min, targeted_geospatial_lat_min, targeted_geospatial_lon_max, targeted_geospatial_lat_max]  # (minx [lon], miny [lat], maxx [lon], maxy [lat])

# define intial time range of interest
targeted_time_range = '2021-01-01T00:00:00Z/2021-01-31T23:59:59Z'

config = {
    'data_dir': '/home/jonas/Documents/Thesis/OceanPlatformControl/data',
    'dataset': dataset_drifter,
    'targeted_bbox': targeted_bbox,
    'area_overlap_type': 'intersects',
    'targeted_time_range': targeted_time_range,
    'usr': "mmariuswiggert",
    "pas": "tamku3-qetroR-guwneq"
}

In [80]:
# Vis location of bbox
x_loc = (targeted_bbox[0]+ targeted_bbox[2])/2
y_loc = (targeted_bbox[1]+ targeted_bbox[3])/2
f = folium.Figure(width=500, height=500)
m = folium.Map(location=[y_loc, x_loc], zoom_start=4).add_to(f)
points = [(targeted_bbox[1], targeted_bbox[0]), (targeted_bbox[3], targeted_bbox[2])]
folium.Rectangle(bounds=points, color='#ff7800').add_to(m)
f

In [2]:
# download and load index files
drifter_data = DrifterData(config, read_only=True)
info = drifter_data.index_data
info.transpose()

...Loading info from: index_platform.txt
...Loading info from: index_latest.txt
...Loading info from: index_monthly.txt
...Loading info from: index_history.txt


In [106]:
targeted_collection = 'monthly'
info = info[info['file_name'].str.contains(targeted_collection)]
info.transposed()

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [4]:
# download all relevant files
nc_links = info["file_name"].tolist()
# for i, file_link in enumerate(nc_links):
#     print(f"downloading {i+1}/{len(nc_links)}: {file_link.split('/')[-1]} ...")
#     drifter_data.downloadNCFile(file_link)

In [10]:
# read in nc file
path2file = os.path.join(config["data_dir"], "drifter_data", "nc_files", file_list[0])
ds = DrifterData.readNCFile(path2file)
ds

<xarray.Dataset>
Dimensions:          (TIME: 744, LATITUDE: 744, LONGITUDE: 744, POSITION: 744,
                      DEPTH: 3)
Coordinates:
  * TIME             (TIME) datetime64[ns] 2021-01-01 ... 2021-01-31T23:00:00
  * LATITUDE         (LATITUDE) float32 18.48 18.47 18.46 ... 17.02 17.02 17.02
  * LONGITUDE        (LONGITUDE) float32 -174.7 -174.7 -174.7 ... 176.2 176.2
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/21)
    TIME_QC          (TIME) float32 ...
    POSITION_QC      (POSITION) float32 ...
    DC_REFERENCE     (TIME) object ...
    DEPH             (TIME, DEPTH) float32 ...
    DEPH_QC          (TIME, DEPTH) float32 ...
    TEMP             (TIME, DEPTH) float64 ...
    ...               ...
    WSPN_MODEL       (TIME, DEPTH) float64 ...
    WSPN_MODEL_QC    (TIME, DEPTH) float32 ...
    WSTE_MODEL       (TIME, DEPTH) float64 ...
    WSTE_MODEL_QC    (TIME, DEPTH) float32 ...
    WSTN_MODEL       (TIME, DEPTH) float64 ...
    WSTN_MODEL_QC    (TIME, DEPTH) float32 ...
Attributes: (12/48)
    platform_code:                  2101528
    platform_name:                  Drifting Buoy
    wmo_platform_code:              2101528
    ices_platform_code:              
    coriolis_platform_code:         2101528
    site_code:                       
    ...                             ...
    id:                             GL_TS_DC_2101528_202101
    netcdf_version:                 netCDF-4 classic model
    format_version:                 1.4
    Conventions:                    CF-1.6 Copernicus-InSituTAC-FormatManual-...
    date_update:                    2021-02-09T06:28:25Z
    history:                        2021-02-09T06:28:25Z : Creation

In [25]:
# read-in files and slice relevant data
file_list = [os.path.join(config["data_dir"], "drifter_data", "nc_files", file_link.split('/')[-1]) for file_link in nc_links]
start_time = datetime(2021,1,1)
end_time = datetime(2021,2,1)

# specify lon and lat mask
mask_lon = (ds.LONGITUDE >= targeted_bbox[0]) & (ds.LONGITUDE <= targeted_bbox[2])
mask_lat = (ds.LATITUDE >= targeted_bbox[1]) & (ds.LATITUDE <= targeted_bbox[3])

# setting up fig
fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
grid_lines = ax.gridlines(draw_labels=True, zorder=5)
grid_lines.top_labels = False
grid_lines.right_labels = False
ax.add_feature(cfeature.LAND, zorder=3, edgecolor='black')

# plotting all drifters on same figure
for file_path in file_list:
    ds = DrifterData.readNCFile(file_path)
    # below select does not work because exact label does not exist -> cannot select
    ds = ds.sel(TIME=slice(start_time, end_time)) #, LONGITUDE=slice(targeted_bbox[0], targeted_bbox[2]), LATITUDE=slice(targeted_bbox[1], targeted_bbox[3]))
    ds = ds.where(mask_lon & mask_lat, 1, 0)
    print(ds)
    u = ds["LONGITUDE"]
    v = ds["LATITUDE"]
    ax.plot(u,v)
plt.show()

: 

: 

In [69]:
# TODO: read in all downloaded files
# cant just read in all from dir, as there are other nc files

ds = xr.open_mfdataset(file_list[:5],
                        combine = "nested",
                        concat_dim = "TIME",
                        preprocess=lambda ds: ds.expand_dims({"model_name": [ds.encoding["source"].split("/")[-1].split(".")[0]]})
                    )

ValueError: arguments without labels along dimension 'POSITION' cannot be aligned because they have different dimension sizes: {16640, 16002, 23300, 23858, 24730}